ATIS DATASET

In [ ]:
pip install datasets

load train data and test data

In [2]:
from datasets import load_dataset
atis_dataset = load_dataset("tuetschek/atis")
print(atis_dataset)
train_data=atis_dataset["train"]
print(train_data)
no_of_samples=len(train_data)
print(no_of_samples)
test_data=atis_dataset["test"]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


atis_train.csv: 0.00B [00:00, ?B/s]

atis_test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/4978 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/893 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'intent', 'text', 'slots'],
        num_rows: 4978
    })
    test: Dataset({
        features: ['id', 'intent', 'text', 'slots'],
        num_rows: 893
    })
})
Dataset({
    features: ['id', 'intent', 'text', 'slots'],
    num_rows: 4978
})
4978


Create a vocabulary for text

In [3]:
unique_vocabulary=set()
for sentence in train_data['text']:
  unique_vocabulary.update(sentence.split())
word_vocab={"<pad>":0, "<unk>":1}
for word in unique_vocabulary:
  word_vocab[word]=len(word_vocab)
print(word_vocab)

{'<pad>': 0, '<unk>': 1, '7': 2, 'originate': 3, 'm': 4, 's': 5, '416': 6, 'companies': 7, 'some': 8, 'seventeenth': 9, '1000': 10, 'o': 11, '718': 12, 'ewr': 13, 'j31': 14, 'so': 15, '402': 16, 'q': 17, 'arrangements': 18, 'limousines': 19, 'repeat': 20, 'offers': 21, 'stands': 22, 'jfk': 23, 'georgia': 24, 'lastest': 25, 'expensive': 26, '420': 27, '1992': 28, 'or': 29, 'california': 30, 'names': 31, 'ninth': 32, 'options': 33, '838': 34, 'after': 35, '2': 36, 'eleventh': 37, 'thirty': 38, 'f28': 39, 'dfw': 40, 'following': 41, '1300': 42, '4': 43, 'francisco': 44, '1110': 45, 'michigan': 46, 'non': 47, 'now': 48, 'me': 49, 'off': 50, 'then': 51, 'transport': 52, 'used': 53, 'capacity': 54, 'landing': 55, 'arriving': 56, 'ff': 57, 'number': 58, 'texas': 59, 'sixteen': 60, '1130': 61, 'fit': 62, '257': 63, 'sorry': 64, 'world': 65, 'airport': 66, 'united': 67, '271': 68, 'detroit': 69, 'takes': 70, '539': 71, 'closest': 72, 'aa': 73, 'snack': 74, 'only': 75, '11': 76, '1220': 77, 'bef

Create a vocabulary for intent

In [4]:
intent_list=list(set(train_data['intent']))
intent_vocab = {'<unk>': 0}
for intent in intent_list:
  intent_vocab[intent] = len(intent_vocab)
print(intent_vocab)
print(len(intent_vocab))

{'<unk>': 0, 'airline+flight_no': 1, 'ground_service': 2, 'flight_time': 3, 'distance': 4, 'abbreviation': 5, 'restriction': 6, 'airport': 7, 'quantity': 8, 'meal': 9, 'flight_no': 10, 'airfare': 11, 'flight+airfare': 12, 'cheapest': 13, 'ground_service+ground_fare': 14, 'aircraft': 15, 'airline': 16, 'aircraft+flight+flight_no': 17, 'airfare+flight_time': 18, 'ground_fare': 19, 'city': 20, 'flight': 21, 'capacity': 22}
23


Create a vocabulary for slot

In [5]:
slots_data=train_data['slots']
slots_labels=[slots.split() for slots in slots_data]
slot_unique_labels=set()
for slots in slots_labels:
  slot_unique_labels.update(slots)
slot_unique_labels=list(slot_unique_labels)
slot_vocab={slot_label:i+2 for i,slot_label in enumerate(slot_unique_labels)}
slot_vocab['<unk>']=1
slot_vocab['<pad>']=0
print(slot_vocab)

{'B-state_name': 2, 'B-class_type': 3, 'B-depart_date.today_relative': 4, 'B-depart_date.day_name': 5, 'B-toloc.airport_code': 6, 'I-airport_name': 7, 'B-return_date.today_relative': 8, 'B-meal_code': 9, 'I-arrive_time.time_relative': 10, 'B-return_date.day_number': 11, 'I-meal_code': 12, 'B-fare_basis_code': 13, 'B-day_name': 14, 'B-arrive_date.today_relative': 15, 'B-toloc.state_code': 16, 'I-arrive_time.time': 17, 'B-depart_time.end_time': 18, 'B-stoploc.city_name': 19, 'B-cost_relative': 20, 'B-depart_time.start_time': 21, 'B-round_trip': 22, 'I-cost_relative': 23, 'B-arrive_date.month_name': 24, 'B-arrive_time.end_time': 25, 'B-return_time.period_mod': 26, 'B-depart_time.time_relative': 27, 'I-round_trip': 28, 'B-fromloc.airport_name': 29, 'B-depart_time.period_of_day': 30, 'I-depart_time.time': 31, 'I-airline_name': 32, 'I-return_date.today_relative': 33, 'B-return_date.day_name': 34, 'I-depart_date.today_relative': 35, 'B-transport_type': 36, 'B-or': 37, 'B-depart_date.month_nam

Create reverse mapping

In [6]:
vocab_id={i:w for w,i in word_vocab.items()}
intent_id={i:w for w,i in intent_vocab.items()}
slot_id={i:w for w,i in slot_vocab.items()}
print(vocab_id)
print(intent_id)
print(slot_id)

{0: '<pad>', 1: '<unk>', 2: '7', 3: 'originate', 4: 'm', 5: 's', 6: '416', 7: 'companies', 8: 'some', 9: 'seventeenth', 10: '1000', 11: 'o', 12: '718', 13: 'ewr', 14: 'j31', 15: 'so', 16: '402', 17: 'q', 18: 'arrangements', 19: 'limousines', 20: 'repeat', 21: 'offers', 22: 'stands', 23: 'jfk', 24: 'georgia', 25: 'lastest', 26: 'expensive', 27: '420', 28: '1992', 29: 'or', 30: 'california', 31: 'names', 32: 'ninth', 33: 'options', 34: '838', 35: 'after', 36: '2', 37: 'eleventh', 38: 'thirty', 39: 'f28', 40: 'dfw', 41: 'following', 42: '1300', 43: '4', 44: 'francisco', 45: '1110', 46: 'michigan', 47: 'non', 48: 'now', 49: 'me', 50: 'off', 51: 'then', 52: 'transport', 53: 'used', 54: 'capacity', 55: 'landing', 56: 'arriving', 57: 'ff', 58: 'number', 59: 'texas', 60: 'sixteen', 61: '1130', 62: 'fit', 63: '257', 64: 'sorry', 65: 'world', 66: 'airport', 67: 'united', 68: '271', 69: 'detroit', 70: 'takes', 71: '539', 72: 'closest', 73: 'aa', 74: 'snack', 75: 'only', 76: '11', 77: '1220', 78: 

Part 1:Intent Classification

Dataset creation

In [7]:
import torch
from torch.utils.data import Dataset
class Intent_Dataset(Dataset):
  def __init__(self,data,vocabulary,intent_list):
    self.data=data
    self.vocabulary=vocabulary
    self.intent_list=intent_list
  def __len__(self):
    return len(self.data)
  def __getitem__(self,idx):
    sentence=self.data[idx]
    text=sentence["text"]
    intent=sentence["intent"]
    text_ids=[self.vocabulary.get(word,self.vocabulary['<unk>']) for word in text.split()]
    intent_id=self.intent_list.get(intent,self.intent_list['<unk>'])
    return{
        "text":torch.tensor(text_ids,dtype=torch.long),
        "intent":torch.tensor(intent_id,dtype=torch.long)
    }

Collate function to pad the texts in a batch to match the same length

In [8]:
from torch.nn.utils.rnn import pad_sequence
def intent_collate_fn(batch):
  texts=[item["text"] for item in batch]
  intent=[item["intent"] for item in batch]
  padded_texts=pad_sequence(texts,batch_first=True,padding_value=0)
  intent_stacked=torch.stack(intent)
  return{
      "text":padded_texts,
      "intent":intent_stacked
  }

Creating the train dataloader

In [9]:
from torch.utils.data import DataLoader
batch_size=30
intent_train_dataset=Intent_Dataset(train_data,word_vocab,intent_vocab)
intent_train_loader=DataLoader(dataset=intent_train_dataset,batch_size=batch_size,shuffle=True,collate_fn=intent_collate_fn)

Implementation of RNN for intent classification

In [10]:
import torch.nn as nn
class Intent_RNN(nn.Module):
  def __init__(self,input_dim,hidden_dim):
    super().__init__()
    self.hidden_dim=hidden_dim
    self.input_to_hidden=nn.Linear(input_dim,hidden_dim)
    self.hidden_to_hidden=nn.Linear(hidden_dim,hidden_dim)
    self.activation=nn.Tanh()
  def forward(self,data):
    batch_size,seq_len,_=data.shape
    outputs=[]
    hidden=torch.zeros(batch_size,self.hidden_dim,device=device)
    for i in range(seq_len):
      input=data[:,i,:]
      hidden=self.activation(self.input_to_hidden(input)+self.hidden_to_hidden(hidden))
      outputs.append(hidden.unsqueeze(1))
    return torch.cat(outputs,dim=1)

Model for intent classifier

In [11]:
class Intent_Classifier(nn.Module):
  def __init__(self,vocab,vocab_size,embed_dim,padding_idx,hidden_dim,output_dim):
    super().__init__()
    self.embedding=nn.Embedding(vocab_size,embed_dim,padding_idx=padding_idx)
    self.RNN=Intent_RNN(embed_dim,hidden_dim)
    self.classification=nn.Linear(hidden_dim,output_dim)
  def forward(self,data):
    embedded=self.embedding(data)
    hidden=self.RNN(embedded)
    pooled_hidden = hidden.mean(dim=1)
    output=self.classification(pooled_hidden)
    return output

Checking if the device is cuda or cpu- cuda preferred for training

In [13]:
import torch
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


Creating models with user-defined parameters

In [14]:
vocab_size=len(word_vocab)
embed_dim=300
padding_idx=word_vocab["<pad>"]
hidden_dim=256
output_dim=len(intent_vocab)
intent_model=Intent_Classifier(word_vocab,vocab_size,embed_dim,padding_idx,hidden_dim,output_dim)
intent_model.to(device)
print(intent_model)

Intent_Classifier(
  (embedding): Embedding(891, 300, padding_idx=0)
  (RNN): Intent_RNN(
    (input_to_hidden): Linear(in_features=300, out_features=256, bias=True)
    (hidden_to_hidden): Linear(in_features=256, out_features=256, bias=True)
    (activation): Tanh()
  )
  (classification): Linear(in_features=256, out_features=23, bias=True)
)


Function to train

In [15]:
def intent_train(model,device,no_of_epochs,train_loader,optimizer,criterion,no_of_samples):
  model.to(device)
  for epoch in range(no_of_epochs):
    model.train()
    total_loss=0
    for batch in train_loader:
      text_batch=batch["text"].to(device)
      label_batch=batch["intent"].to(device)
      optimizer.zero_grad()
      predictions=model(text_batch)
      loss=criterion(predictions,label_batch)
      loss.backward()
      optimizer.step()
      total_loss+=loss.item()
    avg_loss=total_loss/len(train_loader)
    print(f"Epoch {epoch+1}/{no_of_epochs}, Average Loss: {avg_loss:.4f}")

Calling function to train

In [16]:
import torch.optim as optim
import torch.nn as nn
intent_criterion=nn.CrossEntropyLoss()
intent_optimizer=optim.Adam(intent_model.parameters(),lr=0.001)
no_of_epochs=10
intent_train(intent_model,device,no_of_epochs,intent_train_loader,intent_optimizer,intent_criterion,no_of_samples)

Epoch 1/10, Average Loss: 0.9528
Epoch 2/10, Average Loss: 0.6629
Epoch 3/10, Average Loss: 0.4924
Epoch 4/10, Average Loss: 0.3988
Epoch 5/10, Average Loss: 0.2898
Epoch 6/10, Average Loss: 0.2594
Epoch 7/10, Average Loss: 0.2615
Epoch 8/10, Average Loss: 0.2195
Epoch 9/10, Average Loss: 0.1821
Epoch 10/10, Average Loss: 0.1496


Test dataloader

In [17]:
intent_test_dataset=Intent_Dataset(test_data,word_vocab,intent_vocab)
intent_test_loader=DataLoader(dataset=intent_test_dataset,batch_size=batch_size,shuffle=False,collate_fn=intent_collate_fn)

Function to evaluate the model

In [18]:
def intent_evaluation(model,device,test_loader):
  model.eval()
  total_correct = 0
  total_samples=0
  unknown_intent_id=0
  unknown_labels_count=0
  mismatches_to_print=0
  not_flight=0
  with torch.no_grad():
    for batch in test_loader:
      test_batches=batch['text'].to(device)
      label_batches=batch['intent'].to(device)
      predicted_labels=model(test_batches)
      _, predicted_indices = torch.max(predicted_labels, dim=1)
      total_correct += (predicted_indices == label_batches).sum().item()
      total_samples+=label_batches.size(0)
  print(total_correct)
  print(total_samples)
  accuracy = total_correct / total_samples
  print(f"\nModel Accuracy on Test Data: {accuracy * 100:.2f}%")

calling the evaluation function

In [19]:
intent_evaluation(intent_model,device,intent_test_loader)

791
893

Model Accuracy on Test Data: 88.58%


calling the inference function

In [20]:
def inference(model,test_dataset,index,vocab_id,intent_id):
  sample = test_dataset[index]
  input_tensor=sample["text"].unsqueeze(0).to(device)
  model.eval()
  with torch.no_grad():
    logits = model(input_tensor)
    predictions = logits.argmax(dim=-1)
  predicted_labels = predictions.squeeze(0).cpu().tolist()
  text_ids=sample['text'].tolist()
  text_deciphered=[vocab_id[word_id] for word_id in text_ids]
  print("The text is",text_deciphered)
  intent_deciphered=[intent_id[sample['intent'].item()]]
  print("The true value of slot is",intent_deciphered)
  predicted_deciphered=[intent_id[predicted_labels]]
  print("The predicted value of slot is",predicted_deciphered)

In [21]:
index=1
inference(intent_model,intent_test_dataset,index,vocab_id,intent_id)

The text is ['on', 'april', 'first', 'i', 'need', 'a', 'ticket', 'from', 'tacoma', 'to', 'san', 'jose', 'departing', 'before', '7', 'am']
The true value of slot is ['airfare']
The predicted value of slot is ['flight_time']


SLOT Classification

Dataset function for slot filling

In [22]:
from torch.utils.data import Dataset
import torch
class Slot_Dataset(Dataset):
  def __init__(self,data,word_vocab,slot_vocab):
    self.data=data
    self.word_vocab=word_vocab
    self.slot_vocab=slot_vocab
  def __len__(self):
    return len(self.data)
  def __getitem__(self,idx):
    sentence=self.data['text'][idx]
    slots=self.data['slots'][idx]
    word_ids=[self.word_vocab.get(word,self.word_vocab['<unk>']) for word in sentence.split()]
    slot_ids=[self.slot_vocab.get(slot,self.slot_vocab['<unk>']) for slot in slots.split()]
    return{
        "text":torch.tensor(word_ids,dtype=torch.long),
        "slots":torch.tensor(slot_ids,dtype=torch.long)
    }

Collate function to pad text and slot within a batch to make the same length

In [23]:
from torch.nn.utils.rnn import pad_sequence
def slot_collate_fn(batch):
  text_batch=[item["text"] for item in batch]
  slot_batch=[item["slots"] for item in batch]
  padded_text=pad_sequence(text_batch,batch_first=True,padding_value=0)
  padded_slots=pad_sequence(slot_batch,batch_first=True,padding_value=0)
  return{
      "text":padded_text,
      "slots":padded_slots
  }

Creating the dataloader

In [24]:
from torch.utils.data import DataLoader
slot_train_dataset=Slot_Dataset(train_data,word_vocab,slot_vocab)
slot_train_loader=DataLoader(dataset=slot_train_dataset,batch_size=30,shuffle=True,collate_fn=slot_collate_fn)
print(slot_train_loader)

RNN implementation for slot filling

In [25]:
import torch.nn as nn
class SLotRNN(nn.Module):
  def __init__(self,vocab_size,embedding_dim,hidden_dim,output_dim,padding_idx):
    super().__init__()
    self.hidden_dim=hidden_dim
    self.embedding=nn.Embedding(vocab_size,embedding_dim,padding_idx=padding_idx)
    self.input_layer=nn.Linear(embedding_dim,hidden_dim,bias=True)
    self.hidden_layer=nn.Linear(hidden_dim,hidden_dim,bias=True)
    self.activation=nn.Tanh()
    self.output_layer=nn.Linear(hidden_dim,output_dim,bias=True)
  def forward(self,data):
    batch_size,seq_len=data.shape
    embedded_data = self.embedding(data)
    hidden_state=torch.zeros(batch_size,self.hidden_dim,device=data.device)
    outputs = []
    for t in range(seq_len):
      input=embedded_data[:,t,:]
      hidden_state=self.input_layer(input)+self.hidden_layer(hidden_state)
      hidden_state=self.activation(hidden_state)
      output=self.output_layer(hidden_state)
      outputs.append(output.unsqueeze(1))
    return torch.cat(outputs,dim=1)

Checking if the device is cuda or cpu- cuda preferred for training

In [26]:
import torch
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

Creating the model with user-defined parameters

In [27]:
vocab_size=len(word_vocab)
embedding_dim=300
hidden_dim=256
output_dim=len(slot_vocab)
padding_idx=slot_vocab["<pad>"]
slot_model=SLotRNN(vocab_size,embedding_dim,hidden_dim,output_dim,padding_idx)

Function to train the model

In [28]:
def slot_train(model,device,no_of_epochs,train_loader,optimizer,loss_fn,no_of_samples):
  model.to(device)
  no_of_epochs=10
  for epochs in range(no_of_epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
      text=batch['text'].to(device)
      slots=batch['slots'].to(device)
      optimizer.zero_grad()
      predictions=model(text)
      loss=loss_fn(predictions.view(-1,predictions.shape[-1]),slots.view(-1))
      loss.backward()
      optimizer.step()
      total_loss+=loss.item()
    avg_loss=total_loss/no_of_samples
    print(f"Epoch {epochs+1}/{no_of_epochs}, Average Loss: {avg_loss:.4f}")

Training the model

In [29]:
import torch.optim as optim
slot_pad_idx=0
slot_criterion=nn.CrossEntropyLoss(ignore_index=slot_pad_idx)
slot_optimizer=optim.Adam(slot_model.parameters(),lr=0.01)
slot_train(slot_model,device,no_of_epochs,slot_train_loader,slot_optimizer,slot_criterion,no_of_samples)

Epoch 1/10, Average Loss: 0.0106
Epoch 2/10, Average Loss: 0.0042
Epoch 3/10, Average Loss: 0.0034
Epoch 4/10, Average Loss: 0.0030
Epoch 5/10, Average Loss: 0.0034
Epoch 6/10, Average Loss: 0.0027
Epoch 7/10, Average Loss: 0.0025
Epoch 8/10, Average Loss: 0.0027
Epoch 9/10, Average Loss: 0.0027
Epoch 10/10, Average Loss: 0.0028


Creating test_dataloader

In [30]:
from torch.utils.data import DataLoader
slot_test_dataset=Slot_Dataset(train_data,word_vocab,slot_vocab)
slot_test_loader=DataLoader(dataset=slot_test_dataset,batch_size=30,shuffle=False,collate_fn=slot_collate_fn)
print(slot_test_loader)

Function to evaluate

In [31]:
def slot_evaluation(model,device,test_loader):
  model.eval()
  slot_pad_idx=0
  total_correct=0
  total_tokens=0
  with torch.no_grad():
    for batch in test_loader:
      text=batch['text'].to(device)
      slots=batch['slots'].to(device)
      predictions=model(text)
      pred_labels=predictions.argmax(dim=-1)
      mask = (slots != slot_pad_idx)
      total_correct+=((pred_labels==slots)& mask).sum().item()
      total_tokens+= mask.sum().item()
  accuracy = total_correct / total_tokens
  print(f"Accuracy: {accuracy * 100:.2f}%")

Calling evaluation function

In [32]:
slot_evaluation(slot_model,device,slot_test_loader)

Accuracy: 98.22%


Inference function

In [33]:
def slot_inference(model,test_dataset,index,vocab_id,slot_id):
  sample = test_dataset[index]
  input_tensor = sample["text"].unsqueeze(0).to(device)
  model.eval()
  with torch.no_grad():
    logits = model(input_tensor)
    predictions = logits.argmax(dim=-1)
  predicted_labels = predictions.squeeze(0).cpu().tolist()
  text_ids=sample['text'].tolist()
  text_deciphered=[vocab_id[word_id] for word_id in text_ids]
  print("The text is",text_deciphered)
  slot_ids=sample['slots'].tolist()
  slot_deciphered=[slot_id[word_id] for word_id in slot_ids]
  print("The true value of slot is",slot_deciphered)
  predicted_deciphered=[slot_id[word_id] for word_id in predicted_labels]
  print("The predicted value of slot is",predicted_deciphered)

calling inference

In [34]:
index=2
slot_inference(slot_model,slot_test_dataset,index,vocab_id,slot_id)

The text is ['what', 'is', 'the', 'arrival', 'time', 'in', 'san', 'francisco', 'for', 'the', '755', 'am', 'flight', 'leaving', 'washington']
The true value of slot is ['O', 'O', 'O', 'B-flight_time', 'I-flight_time', 'O', 'B-fromloc.city_name', 'I-fromloc.city_name', 'O', 'O', 'B-depart_time.time', 'I-depart_time.time', 'O', 'O', 'B-fromloc.city_name']
The predicted value of slot is ['O', 'O', 'O', 'B-flight_time', 'I-flight_time', 'O', 'B-toloc.city_name', 'I-stoploc.city_name', 'O', 'O', 'B-depart_time.time', 'I-depart_time.time', 'O', 'O', 'B-fromloc.city_name']


Slot-To-Intent

Function to get the predicted labels of slot based on text

In [43]:
def slot_pred(model,sentence):
  input_tensor = sentence.unsqueeze(0).to(device)
  model.eval()
  with torch.no_grad():
    logits = model(input_tensor)
    predictions = logits.argmax(dim=-1)
  predicted_labels = predictions.squeeze(0).cpu().tolist()
  return predicted_labels

Dataset creation including predicted slot values

In [35]:
from torch.utils.data import Dataset
class Slot_to_Intent_Dataset(Dataset):
  def __init__(self,data,model,word_vocab,intent_vocab):
    self.data=data
    self.word_vocab=word_vocab
    self.intent_vocab=intent_vocab
    self.slot_vocab=slot_vocab
    self.model=model
  def __len__(self):
    return len(self.data)
  def __getitem__(self,idx):
    sentence=self.data['text'][idx]
    intent=self.data['intent'][idx]
    text_ids=[self.word_vocab.get(word,self.word_vocab['<unk>']) for word in sentence.split()]
    intent_id=self.intent_vocab.get(intent,self.intent_vocab['<unk>'])
    text_tensor = torch.tensor(text_ids, dtype=torch.long)
    slot_predicted_ids=slot_pred(self.model,text_tensor)
    return{
        "text":torch.tensor(text_ids,dtype=torch.long),
        "intent":torch.tensor(intent_id,dtype=torch.long),
        "slot":torch.tensor(slot_predicted_ids,dtype=torch.long)
    }

Collate function

In [36]:
from torch.nn.utils.rnn import pad_sequence
def slot_to_intent_collate_fn(batch):
  texts=[item["text"] for item in batch]
  intent=[item["intent"] for item in batch]
  slot=[item["slot"] for item in batch]
  padded_texts=pad_sequence(texts,batch_first=True,padding_value=0)
  padded_slots=pad_sequence(slot,batch_first=True,padding_value=0)
  intent_stacked=torch.stack(intent)
  return{
      "text":padded_texts,
      "intent":intent_stacked,
      "slot":padded_slots
  }

creating dataloader and dataset

In [37]:
from torch.utils.data import DataLoader
batch_size=30
slot_to_intent_train_dataset=Slot_to_Intent_Dataset(train_data,slot_model,word_vocab,intent_vocab)
slot_to_intent_train_loader=DataLoader(dataset=slot_to_intent_train_dataset,batch_size=batch_size,shuffle=True,collate_fn=slot_to_intent_collate_fn)

Creating models

In [38]:
import torch.nn as nn
class Slot_to_Intent_RNN(nn.Module):
  def __init__(self,input_dim,hidden_dim):
    super().__init__()
    self.hidden_dim=hidden_dim
    self.input_to_hidden=nn.Linear(input_dim,hidden_dim)
    self.hidden_to_hidden=nn.Linear(hidden_dim,hidden_dim)
    self.activation=nn.Tanh()
  def forward(self,data):
    outputs=[]
    batch_size,seq_len,_=data.shape
    hidden=torch.zeros(batch_size,self.hidden_dim,device=device)
    for i in range(seq_len):
      input=data[:,i,:]
      hidden=self.activation(self.input_to_hidden(input)+self.hidden_to_hidden(hidden))
      outputs.append(hidden.unsqueeze(1))
    return torch.cat(outputs,dim=1)

In [39]:
class SLot_to_Intent_Classifier(nn.Module):
  def __init__(self,vocab,vocab_size,embed_dim,padding_idx,hidden_dim,output_dim,slot_vocab_size,slot_embed_dim,slot_padding_idx):
    super().__init__()
    self.embedding=nn.Embedding(vocab_size,embed_dim,padding_idx=padding_idx)
    self.RNN=Slot_to_Intent_RNN(embed_dim,hidden_dim)
    self.classification=nn.Linear(hidden_dim+slot_embed_dim,output_dim)
    self.slot_embedding = nn.Embedding(slot_vocab_size, slot_embed_dim, padding_idx=slot_padding_idx)
  def forward(self,data,slot_pred):
    embedded=self.embedding(data)
    hidden=self.RNN(embedded)
    slot_embeds = self.slot_embedding(slot_pred)
    slot_features = slot_embeds.mean(dim=1)
    pooled_hidden = hidden.mean(dim=1)
    combined = torch.cat([pooled_hidden, slot_features], dim=1)
    output=self.classification(combined)
    return output

Creating model with user-defined parameters

In [40]:
vocab_size=len(word_vocab)
slot_vocab_size=len(slot_vocab)
slot_embed_dim=32
slot_padding_idx=0
embed_dim=300
padding_idx=word_vocab["<pad>"]
hidden_dim=256
output_dim=len(intent_vocab)
slot_to_intent_model=SLot_to_Intent_Classifier(word_vocab,vocab_size,embed_dim,padding_idx,hidden_dim,output_dim,slot_vocab_size,slot_embed_dim,slot_padding_idx)
slot_to_intent_model.to(device)
print(slot_to_intent_model)

SLot_to_Intent_Classifier(
  (embedding): Embedding(891, 300, padding_idx=0)
  (RNN): Slot_to_Intent_RNN(
    (input_to_hidden): Linear(in_features=300, out_features=256, bias=True)
    (hidden_to_hidden): Linear(in_features=256, out_features=256, bias=True)
    (activation): Tanh()
  )
  (classification): Linear(in_features=288, out_features=23, bias=True)
  (slot_embedding): Embedding(125, 32, padding_idx=0)
)


train function

In [41]:
def slot_to_intent_train(model,device,no_of_epochs,train_loader,optimizer,criterion,no_of_samples):
  model.to(device)
  for epoch in range(no_of_epochs):
    model.train()
    total_loss=0
    for batch in train_loader:
      text_batch=batch["text"].to(device)
      label_batch=batch["intent"].to(device)
      slot_batch=batch["slot"].to(device)
      optimizer.zero_grad()
      predictions=model(text_batch,slot_batch)
      loss=criterion(predictions,label_batch)
      loss.backward()
      optimizer.step()
      total_loss+=loss.item()
    avg_loss=total_loss/len(train_loader)
    print(f"Epoch {epoch+1}/{no_of_epochs}, Average Loss: {avg_loss:.4f}")

calling the train function

In [44]:
import torch.optim as optim
slot_to_intent_criterion=nn.CrossEntropyLoss()
slot_to_intent_optimizer=optim.Adam(slot_to_intent_model.parameters(),lr=0.001)
no_of_epochs=10
no_of_samples=len(train_data)
print(no_of_samples)
slot_to_intent_train(slot_to_intent_model,device, no_of_epochs, slot_to_intent_train_loader, slot_to_intent_optimizer, slot_to_intent_criterion, no_of_samples)

4978
Epoch 1/10, Average Loss: 0.9389
Epoch 2/10, Average Loss: 0.5385
Epoch 3/10, Average Loss: 0.4842
Epoch 4/10, Average Loss: 0.3977
Epoch 5/10, Average Loss: 0.2907
Epoch 6/10, Average Loss: 0.3218
Epoch 7/10, Average Loss: 0.2939
Epoch 8/10, Average Loss: 0.3168
Epoch 9/10, Average Loss: 0.2032
Epoch 10/10, Average Loss: 0.1496


Test dataloader

In [45]:
slot_to_intent_test_dataset=Slot_to_Intent_Dataset(test_data,slot_model,word_vocab,intent_vocab)
slot_to_intent_test_loader=DataLoader(dataset=slot_to_intent_test_dataset,batch_size=batch_size,shuffle=True,collate_fn=slot_to_intent_collate_fn)

Evaluation function

In [46]:
def slot_to_intent_evaluation(model,device,test_loader):
  model.eval()
  total_correct = 0
  total_samples=0
  unknown_intent_id=0
  unknown_labels_count=0
  mismatches_to_print=0
  not_flight=0
  with torch.no_grad():
    for batch in test_loader:
      test_batches=batch['text'].to(device)
      label_batches=batch['intent'].to(device)
      slot_batches=batch['slot'].to(device)
      predicted_labels=model(test_batches,slot_batches)
      _, predicted_indices = torch.max(predicted_labels, dim=1)
      total_correct += (predicted_indices == label_batches).sum().item()
      total_samples+=label_batches.size(0)
  print(total_correct)
  print(total_samples)
  accuracy = total_correct / total_samples
  print(f"\nModel Accuracy on Test Data: {accuracy * 100:.2f}%")

In [47]:
slot_to_intent_evaluation(slot_to_intent_model,device,slot_to_intent_test_loader)

799
893

Model Accuracy on Test Data: 89.47%


Inference function

In [48]:
def slot_to_inference(model,test_dataset,index,vocab_id,intent_id):
  sample = test_dataset[index]
  input_tensor=sample["text"].unsqueeze(0).to(device)
  slot_tensor=sample["slot"].unsqueeze(0).to(device)
  model.eval()
  with torch.no_grad():
    logits = model(input_tensor,slot_tensor)
    predictions = logits.argmax(dim=-1)
  predicted_labels = predictions.squeeze(0).cpu().tolist()
  text_ids=sample['text'].tolist()
  text_deciphered=[vocab_id[word_id] for word_id in text_ids]
  print("The text is",text_deciphered)
  intent_deciphered=[intent_id[sample['intent'].item()]]
  print("The true value of slot is",intent_deciphered)
  predicted_deciphered=[intent_id[predicted_labels]]
  print("The predicted value of slot is",predicted_deciphered)

In [49]:
index=1
slot_to_inference(slot_to_intent_model,slot_to_intent_test_dataset,index,vocab_id,intent_id)

The text is ['on', 'april', 'first', 'i', 'need', 'a', 'ticket', 'from', 'tacoma', 'to', 'san', 'jose', 'departing', 'before', '7', 'am']
The true value of slot is ['airfare']
The predicted value of slot is ['airfare']


Intent to Slot

Function to get the predicted labels of slot based on text

In [50]:
def intent_pred(model,sentence):
  input_tensor=sentence.unsqueeze(0).to(device)
  model.eval()
  with torch.no_grad():
    logits = model(input_tensor)
    predictions = logits.argmax(dim=-1)
  predicted_labels = predictions.squeeze(0).cpu().tolist()
  return predicted_labels

Dataset Creation

In [51]:
from torch.utils.data import Dataset
class Intent_to_Slot_Dataset(Dataset):
  def __init__(self,data,model,word_vocab,slot_vocab):
    self.data=data
    self.word_vocab=word_vocab
    self.slot_vocab=slot_vocab
    self.model=model
  def __len__(self):
    return len(self.data)
  def __getitem__(self,idx):
    sentence=self.data['text'][idx]
    slots=self.data['slots'][idx]
    text_ids=[self.word_vocab.get(word,self.word_vocab['<unk>']) for word in sentence.split()]
    slot_ids=[self.slot_vocab.get(slot,self.slot_vocab['<unk>']) for slot in slots.split()]
    text_tensor = torch.tensor(text_ids, dtype=torch.long)
    slot_tensor=torch.tensor(slot_ids,dtype=torch.long)
    intent_pred_ids=intent_pred(self.model,text_tensor)
    intent_tensor=torch.tensor(intent_pred_ids, dtype=torch.long)
    return{
        "text":text_tensor,
        "intent":intent_tensor,
        "slot":slot_tensor
    }

Collate function

In [52]:
from torch.nn.utils.rnn import pad_sequence
def intent_to_slot_collate_fn(batch):
  texts=[item["text"] for item in batch]
  intent=[item["intent"] for item in batch]
  slot=[item["slot"] for item in batch]
  padded_texts=pad_sequence(texts,batch_first=True,padding_value=0)
  padded_slots=pad_sequence(slot,batch_first=True,padding_value=0)
  intent_stacked=torch.stack(intent)
  return{
      "text":padded_texts,
      "intent":intent_stacked,
      "slot":padded_slots
  }

Dataloader creation

In [53]:
from torch.utils.data import DataLoader
batch_size=30
intent_to_slot_train_dataset=Intent_to_Slot_Dataset(train_data,intent_model,word_vocab,slot_vocab)
intent_to_slot_train_loader=DataLoader(dataset=intent_to_slot_train_dataset,batch_size=batch_size,shuffle=True,collate_fn=intent_to_slot_collate_fn)

Classification model

In [54]:
import torch.nn as nn
class Intent_to_slot(nn.Module):
  def __init__(self,vocab_size,embedding_dim,hidden_dim,output_dim,padding_idx,intent_embed_dim,intent_vocab_size):
    super().__init__()
    self.hidden_dim=hidden_dim
    self.embedding=nn.Embedding(vocab_size,embedding_dim,padding_idx=padding_idx)
    self.input_layer=nn.Linear(embedding_dim,hidden_dim,bias=True)
    self.hidden_layer=nn.Linear(hidden_dim,hidden_dim,bias=True)
    self.intent_embedding = nn.Embedding(intent_vocab_size, intent_embed_dim)
    self.activation=nn.Tanh()
    self.output_layer=nn.Linear(hidden_dim+intent_embed_dim,output_dim,bias=True)
  def forward(self,data,intent):
    batch_size,seq_len=data.shape
    embedded_data = self.embedding(data)
    hidden_state=torch.zeros(batch_size,self.hidden_dim,device=data.device)
    outputs = []
    intent_embed = self.intent_embedding(intent)
    for t in range(seq_len):
      input=embedded_data[:,t,:]
      hidden_state=self.input_layer(input)+self.hidden_layer(hidden_state)
      hidden_state=self.activation(hidden_state)
      combined = torch.cat([hidden_state, intent_embed], dim=1)
      output=self.output_layer(combined)
      outputs.append(output.unsqueeze(1))
    return torch.cat(outputs,dim=1)

creating model with user-defined parameters

In [55]:
vocab_size=len(word_vocab)
embedding_dim=300
hidden_dim=256
output_dim=len(slot_vocab)
padding_idx=0
intent_embed_dim=32
intent_vocab_size=len(intent_vocab)
intent_to_slot_model=Intent_to_slot(vocab_size,embedding_dim,hidden_dim,output_dim,padding_idx,intent_embed_dim,intent_vocab_size)
intent_to_slot_model.to(device)
print(intent_to_slot_model)

Intent_to_slot(
  (embedding): Embedding(891, 300, padding_idx=0)
  (input_layer): Linear(in_features=300, out_features=256, bias=True)
  (hidden_layer): Linear(in_features=256, out_features=256, bias=True)
  (intent_embedding): Embedding(23, 32)
  (activation): Tanh()
  (output_layer): Linear(in_features=288, out_features=125, bias=True)
)


Training function

In [56]:
def intent_to_slot_train(model,device,no_of_epochs,train_loader,optimizer,loss_fn,no_of_samples):
  model.to(device)
  for epochs in range(no_of_epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
      text=batch['text'].to(device)
      intent=batch['intent'].to(device)
      slots=batch['slot'].to(device)
      optimizer.zero_grad()
      predictions=model(text,intent)
      loss=loss_fn(predictions.view(-1,predictions.shape[-1]),slots.view(-1))
      loss.backward()
      optimizer.step()
      total_loss+=loss.item()
    avg_loss=total_loss/no_of_samples
    print(f"Epoch {epochs+1}/{no_of_epochs}, Average Loss: {avg_loss:.4f}")

In [58]:
import torch.optim as optim
slot_pad_idx=slot_vocab["<pad>"]
intent_to_slot_criterion=nn.CrossEntropyLoss(ignore_index=slot_pad_idx)
intent_to_slot_optimizer=optim.Adam(intent_to_slot_model.parameters(),lr=0.01)
no_of_epochs=10
no_of_samples=len(train_data)
print(no_of_samples)
intent_to_slot_train(intent_to_slot_model,device,no_of_epochs,intent_to_slot_train_loader, intent_to_slot_optimizer,intent_to_slot_criterion, no_of_samples)

4978
Epoch 1/10, Average Loss: 0.0050
Epoch 2/10, Average Loss: 0.0037
Epoch 3/10, Average Loss: 0.0029
Epoch 4/10, Average Loss: 0.0030
Epoch 5/10, Average Loss: 0.0027
Epoch 6/10, Average Loss: 0.0027
Epoch 7/10, Average Loss: 0.0031
Epoch 8/10, Average Loss: 0.0031
Epoch 9/10, Average Loss: 0.0031
Epoch 10/10, Average Loss: 0.0031


Test dataloader

In [64]:
intent_to_slot_test_dataset=Intent_to_Slot_Dataset(test_data,intent_model,word_vocab,slot_vocab)
intent_to_slot_test_loader=DataLoader(dataset=intent_to_slot_test_dataset,batch_size=batch_size,shuffle=True,collate_fn=intent_to_slot_collate_fn)

evaluation function

In [65]:
def intent_to_slot_evaluation(model,device,test_loader):
  model.eval()
  slot_pad_idx=0
  total_correct=0
  total_tokens=0
  with torch.no_grad():
    for batch in test_loader:
      text=batch['text'].to(device)
      intent=batch['intent'].to(device)
      slots=batch['slot'].to(device)
      predictions=model(text,intent)
      pred_labels=predictions.argmax(dim=-1)
      mask = (slots != slot_pad_idx)
      total_correct+=((pred_labels==slots)& mask).sum().item()
      total_tokens+= mask.sum().item()
  accuracy = total_correct / total_tokens
  print(f"Accuracy: {accuracy * 100:.2f}%")

In [66]:
intent_to_slot_evaluation(intent_to_slot_model,device,intent_to_slot_test_loader)

Accuracy: 95.46%


Intent-to-Slot inference

In [67]:
def intent_to_slot_inference(model,test_dataset,index,vocab_id,slot_id):
  sample = test_dataset[index]
  input_tensor = sample["text"].unsqueeze(0).to(device)
  intent_tensor=sample['intent'].unsqueeze(0).to(device)
  model.eval()
  with torch.no_grad():
    logits = model(input_tensor,intent_tensor)
    predictions = logits.argmax(dim=-1)
  predicted_labels = predictions.squeeze(0).cpu().tolist()
  text_ids=sample['text'].tolist()
  text_deciphered=[vocab_id[word_id] for word_id in text_ids]
  print("The text is",text_deciphered)
  slot_ids=sample['slot'].tolist()
  slot_deciphered=[slot_id[word_id] for word_id in slot_ids]
  print("The true value of slot is",slot_deciphered)
  predicted_deciphered=[slot_id[word_id] for word_id in predicted_labels]
  print("The predicted value of slot is",predicted_deciphered)

In [68]:
index=2
intent_to_slot_inference(intent_to_slot_model,intent_to_slot_test_dataset,index,vocab_id,slot_id)

The text is ['on', 'april', 'first', 'i', 'need', 'a', 'flight', 'going', 'from', 'phoenix', 'to', 'san', 'diego']
The true value of slot is ['O', 'B-depart_date.month_name', 'B-depart_date.day_number', 'O', 'O', 'O', 'O', 'O', 'O', 'B-fromloc.city_name', 'O', 'B-toloc.city_name', 'I-toloc.city_name']
The predicted value of slot is ['O', 'B-depart_date.month_name', 'B-depart_date.day_number', 'O', 'O', 'O', 'O', 'O', 'O', 'B-fromloc.city_name', 'O', 'B-toloc.city_name', 'I-toloc.city_name']


Joint-Model

Dataset Creation

In [69]:
from torch.utils.data import Dataset
import torch
class Joint_Dataset(Dataset):
  def __init__(self,data,word_vocab,slot_vocab,intent_vocab):
    self.data=data
    self.word_vocab=word_vocab
    self.slot_vocab=slot_vocab
  def __len__(self):
    return len(self.data)
  def __getitem__(self,idx):
    sentence=self.data['text'][idx]
    slots=self.data['slots'][idx]
    intent=self.data['intent'][idx]
    word_ids=[self.word_vocab.get(word,self.word_vocab['<unk>']) for word in sentence.split()]
    slot_ids=[self.slot_vocab.get(slot,self.slot_vocab['<unk>']) for slot in slots.split()]
    intent_id=intent_vocab.get(intent,intent_vocab['<unk>'])
    return{
        "text":torch.tensor(word_ids,dtype=torch.long),
        "slots":torch.tensor(slot_ids,dtype=torch.long),
        "intent":torch.tensor(intent_id,dtype=torch.long)
    }

collate function

In [70]:
from torch.nn.utils.rnn import pad_sequence
def joint_collate_fn(batch):
  text_batch=[item["text"] for item in batch]
  slot_batch=[item["slots"] for item in batch]
  intent_batch=[item["intent"] for item in batch]
  padded_text=pad_sequence(text_batch,batch_first=True,padding_value=0)
  padded_slots=pad_sequence(slot_batch,batch_first=True,padding_value=0)
  intent=torch.stack(intent_batch)
  return{
      "text":padded_text,
      "slots":padded_slots,
      "intent":intent
  }

train dataloader

In [71]:
from torch.utils.data import DataLoader
batch_size=30
joint_train_dataset=Joint_Dataset(train_data,word_vocab,slot_vocab,intent_vocab)
joint_train_loader=DataLoader(dataset=joint_train_dataset,batch_size=batch_size,shuffle=True,collate_fn=joint_collate_fn)

Model creation

In [72]:
import torch.nn as nn
class JointRnn(nn.Module):
  def __init__(self,input_dim,hidden_dim):
    super().__init__()
    self.hidden_dim=hidden_dim
    self.input_to_hidden=nn.Linear(input_dim,hidden_dim)
    self.hidden_to_hidden=nn.Linear(hidden_dim,hidden_dim)
    self.activation=nn.Tanh()
  def forward(self,data):
    batch_size,seq_len,_=data.size()
    outputs=[]
    hidden=torch.zeros(batch_size, self.hidden_dim, device=data.device)
    for i in range(seq_len):
      input=data[:,i,:]
      hidden=self.activation(self.input_to_hidden(input)+self.hidden_to_hidden(hidden))
      outputs.append(hidden.unsqueeze(1))
    return torch.cat(outputs, dim=1)

In [73]:
class Joint_Model(nn.Module):
  def __init__(self,vocab_size,embedding_dim,hidden_dim,padding_idx,intent_num,slot_num):
    super().__init__()
    self.embedding=nn.Embedding(vocab_size,embedding_dim,padding_idx=padding_idx)
    self.rnn=JointRnn(embedding_dim,hidden_dim)
    self.intent_classifier=nn.Linear(hidden_dim,intent_num)
    self.slot_classifier=nn.Linear(hidden_dim,slot_num)
  def forward(self,data):
    embedded=self.embedding(data)
    rnn_output=self.rnn(embedded)
    pooled_hidden=rnn_output.mean(dim=1)
    intent_output=self.intent_classifier(pooled_hidden)
    slot_output=self.slot_classifier(rnn_output)
    return intent_output,slot_output

Creating a model with user defined parameters

In [74]:
vocab_size=len(word_vocab)
embed_dim=300
padding_idx=word_vocab["<pad>"]
hidden_dim=256
intent_num=len(intent_vocab)
slot_num=len(slot_vocab)
model=Joint_Model(vocab_size,embed_dim,hidden_dim,padding_idx,intent_num,slot_num)
model.to(device)
print(model)

Joint_Model(
  (embedding): Embedding(891, 300, padding_idx=0)
  (rnn): JointRnn(
    (input_to_hidden): Linear(in_features=300, out_features=256, bias=True)
    (hidden_to_hidden): Linear(in_features=256, out_features=256, bias=True)
    (activation): Tanh()
  )
  (intent_classifier): Linear(in_features=256, out_features=23, bias=True)
  (slot_classifier): Linear(in_features=256, out_features=125, bias=True)
)


Trainign function

In [78]:
def train_joint_model(model,no_of_epochs,optimizer,intent_criterion,slot_criterion,joint_train_loader):
  model.to(device)
  for epoch in range(no_of_epochs):
    model.train()
    total_loss=0
    total_slot_loss=0
    total_intent_loss=0
    for batch in joint_train_loader:
      text_batch=batch["text"].to(device)
      slot_batch=batch["slots"].to(device)
      intent_batch=batch["intent"].to(device)
      optimizer.zero_grad()
      intent_output,slot_output=model(text_batch)
      intent_loss=intent_criterion(intent_output,intent_batch)
      slot_loss=slot_criterion(slot_output.view(-1,slot_output.shape[-1]),slot_batch.view(-1))
      loss=intent_loss+slot_loss
      loss.backward()
      optimizer.step()
      total_loss+=loss.item()
      total_slot_loss+=slot_loss.item()
      total_intent_loss+=intent_loss.item()
    avg_loss = total_loss / len(joint_train_loader)
    avg_slot_loss = total_slot_loss / len(joint_train_loader)
    avg_intent_loss = total_intent_loss / len(joint_train_loader)
    print(f"Epoch {epoch+1}/{no_of_epochs}, Total Loss: {avg_loss:.4f} , Slot Loss: {avg_slot_loss:.4f} , Intent Loss: {avg_intent_loss:.4f}")

In [79]:
import torch.optim as optim
import torch.nn as nn
slot_pad_idx=0
optimizer = optim.Adam(model.parameters(), lr=0.0005)
slot_criterion = nn.CrossEntropyLoss(ignore_index=slot_pad_idx)
intent_criterion = nn.CrossEntropyLoss()
no_of_epochs=10
train_joint_model(model,no_of_epochs,optimizer,intent_criterion,slot_criterion,joint_train_loader)

Epoch 1/10, Total Loss: 1.0353 , Slot Loss: 0.3195 , Intent Loss: 0.7157
Epoch 2/10, Total Loss: 0.7633 , Slot Loss: 0.2104 , Intent Loss: 0.5529
Epoch 3/10, Total Loss: 0.6269 , Slot Loss: 0.1663 , Intent Loss: 0.4606
Epoch 4/10, Total Loss: 0.4937 , Slot Loss: 0.1373 , Intent Loss: 0.3564
Epoch 5/10, Total Loss: 0.3763 , Slot Loss: 0.1161 , Intent Loss: 0.2601
Epoch 6/10, Total Loss: 0.3669 , Slot Loss: 0.1033 , Intent Loss: 0.2636
Epoch 7/10, Total Loss: 0.3825 , Slot Loss: 0.1019 , Intent Loss: 0.2806
Epoch 8/10, Total Loss: 0.3761 , Slot Loss: 0.0919 , Intent Loss: 0.2842
Epoch 9/10, Total Loss: 0.3340 , Slot Loss: 0.0858 , Intent Loss: 0.2482
Epoch 10/10, Total Loss: 0.3718 , Slot Loss: 0.0804 , Intent Loss: 0.2914


test dataloader

In [80]:
joint_test_dataset=Joint_Dataset(test_data,word_vocab,slot_vocab,intent_vocab)
joint_test_dataloader=DataLoader(dataset=joint_test_dataset,batch_size=batch_size,shuffle=True,collate_fn=joint_collate_fn)

Evaluation function

In [81]:
def joint_evaluation(model,device,test_loader):
  model.eval()
  slot_pad_idx=0
  correct_intent=0
  total_intent=0
  correct_slots=0
  total_slots=0
  with torch.no_grad():
    for batch in test_loader:
      text=batch['text'].to(device)
      slots=batch['slots'].to(device)
      intent=batch['intent'].to(device)
      intent_output,slot_output=model(text)
      intent_preds = intent_output.argmax(dim=1)
      correct_intent += (intent_preds == intent).sum().item()
      total_intent += intent_preds.size(0)
      slot_preds = slot_output.argmax(dim=-1)
      mask = (slots!= slot_pad_idx)
      correct_slots += ((slot_preds == slots) & mask).sum().item()
      total_slots += mask.sum().item()
  intent_acc = correct_intent / total_intent
  slot_acc = correct_slots / total_slots
  print(f"Intent Accuracy: {intent_acc * 100:.2f}%")
  print(f"Slot Accuracy: {slot_acc * 100:.2f}%")

In [82]:
joint_evaluation(model,device,joint_test_dataloader)

Intent Accuracy: 86.45%
Slot Accuracy: 95.76%


inference function

In [83]:
def joint_inference(model, test_dataset, index, vocab_id, intent_id, slot_id, device):
    sample = test_dataset[index]
    input_tensor = sample["text"].unsqueeze(0).to(device)
    true_intent = sample["intent"].item()
    true_slots = sample["slots"].tolist()
    model.eval()
    with torch.no_grad():
        intent_output, slot_output = model(input_tensor)
        intent_pred = intent_output.argmax(dim=1).item()
        slot_pred = slot_output.argmax(dim=-1).squeeze(0).cpu().tolist()
    text_ids = sample['text'].tolist()
    text_deciphered = [vocab_id[word_id] for word_id in text_ids]
    print("The text is:", text_deciphered)
    true_intent_deciphered = intent_id[true_intent]
    pred_intent_deciphered = intent_id[intent_pred]
    print("True intent:", true_intent_deciphered)
    print("Predicted intent:", pred_intent_deciphered)
    true_slots_deciphered = [slot_id[slot] for slot in true_slots]
    pred_slots_deciphered = [slot_id[slot] for slot in slot_pred]
    print("True slots:", true_slots_deciphered)
    print("Predicted slots:", pred_slots_deciphered)

In [85]:
index=1
joint_inference(model,joint_test_dataset,index,vocab_id,intent_id,slot_id,device)

The text is: ['on', 'april', 'first', 'i', 'need', 'a', 'ticket', 'from', 'tacoma', 'to', 'san', 'jose', 'departing', 'before', '7', 'am']
True intent: airfare
Predicted intent: flight
True slots: ['O', 'B-depart_date.month_name', 'B-depart_date.day_number', 'O', 'O', 'O', 'O', 'O', 'B-fromloc.city_name', 'O', 'B-toloc.city_name', 'I-toloc.city_name', 'O', 'B-depart_time.time_relative', 'B-depart_time.time', 'I-depart_time.time']
Predicted slots: ['O', 'B-depart_date.month_name', 'B-depart_date.day_number', 'O', 'O', 'O', 'O', 'O', 'B-fromloc.city_name', 'O', 'B-toloc.city_name', 'I-toloc.city_name', 'O', 'B-depart_time.time_relative', 'B-depart_time.time', 'I-depart_time.time']


SLURP DATASET

uploading train file

In [86]:
from google.colab import files
slurp_dataset=files.upload()

Saving train.jsonl to train.jsonl


In [90]:
def slot_generation(data):
  words = [token['surface'] for token in data['tokens']]
  no_of_tokens=len(words)
  slots=["O"]*no_of_tokens
  for slot in data['entities']:
    for idx in slot['span']:
      slots[idx]=slot['type']
  intent=data['intent']
  return words,slots,intent

make train data

In [87]:
import io
import json
data_t=[]
slurp_file_content=slurp_dataset['train.jsonl'].decode('utf-8')
slurp_file=io.StringIO(slurp_file_content)
for line in  slurp_file:
    data_t.append(json.loads(line))

In [92]:
train_data={}
train_data['text']=[]
train_data['slot']=[]
train_data['intent']=[]
for i in range (len(data_t)):
  words,slots,intent=slot_generation(data_t[i])
  train_data['text'].append(words)
  train_data['slot'].append(slots)
  train_data['intent'].append(intent)
no_of_samples=len(train_data['text'])
print(no_of_samples)

11514


In [94]:
print(train_data['text'][1])
print(train_data['slot'][1])
print(train_data['intent'][1])

['i', 'need', 'an', 'event', 'three', 'days', 'from', 'now', 'scheduled', 'with', 'amy']
['O', 'O', 'O', 'O', 'date', 'date', 'date', 'date', 'O', 'O', 'person']
calendar_set


make test dataset

In [98]:
from google.colab import files
slurp_dataset=files.upload()

Saving test.jsonl to test.jsonl


In [99]:
import io
import json
data_te=[]
slurp_file_content=slurp_dataset['test.jsonl'].decode('utf-8')
slurp_file=io.StringIO(slurp_file_content)
for line in  slurp_file:
    data_te.append(json.loads(line))

In [100]:
test_data={}
test_data['text']=[]
test_data['slot']=[]
test_data['intent']=[]
for i in range (len(data_te)):
  words,slots,intent=slot_generation(data_te[i])
  test_data['text'].append(words)
  test_data['slot'].append(slots)
  test_data['intent'].append(intent)
no_of_test_samples=len(test_data['text'])
print(no_of_test_samples)

2974


In [101]:
print(test_data['text'][1])
print(test_data['slot'][1])
print(test_data['intent'][1])

['put', 'meeting', 'with', 'pawel', 'for', 'tomorrow', 'ten', 'am']
['O', 'event_name', 'O', 'person', 'O', 'date', 'time', 'time']
calendar_set
